In [11]:
import librosa
import numpy as np
from google.colab import drive
import os
import json
import math

In [12]:
#Frecuencia en la cual se tomaran muestras en un audio
frecuencia_muestra = 22050
duracion_audio = 30 #En segundos
num_de_muestras_por_audio = frecuencia_muestra * duracion_audio

In [13]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
folder_path = "/content/drive/My Drive/Machine Learning TP/Datasets/Audio/Data/genres_original" #path a la carpeta de los audios.
json_path = "/content/drive/My Drive/Machine Learning TP/Datasets/Audio/jsondataaudio.json" #path al archivo json donde guardar los datos extraidos de los audios

Este paso se hace primero para obtener un json con la informacion de los audios, ya que realizar la operacion de extraccion de datos cuesta tiempo, entonces se extrae una vez y luego se usa.

In [15]:
def save_mfcc(path_datos, json_path, n_mfcc = 13, n_fft = 2048, hop_length = 512, num_segmentos = 10):
  
  #diccionario para almacenar datos
  data = {
      "generos": [], #Esto tendra el genero de las canciones
      "mfcc": [], #Seria el input train, tendra todos los valores del mfcc
      "labels": [] #Seria el output, esta relacionado con a que genero representa un mfcc
  }

  
  num_de_muestras_por_segmento = int(num_de_muestras_por_audio / num_segmentos) 
  
  #Como necesitamos que todos los valores de entrenamiento del modelo tengan la misma forma
  #definimos un numero fijo de vectores mfcc a utilizar.
  num_de_vectores_mfcc_por_segmento = math.ceil(num_de_muestras_por_segmento / hop_length)

  #dirpath: el path del directorio actual
  #dirname: el nombre del directorio
  #filenames: los nombres de los archivos del directorio
  #indice tiene el numero de la iteracion, que representara el genero
  for indice, (dirpath, dirnames, filenames) in enumerate(os.walk(folder_path)):

    #No queremos estar en la carpeta principal, sino en la de los generos que tiene los audiso
    if dirpath is not folder_path:

      #separamos el path por cada / en el path en un array y agarramos el ultimo, que seria el genero 
      folder_genero = dirpath.split("/")[-1]
      data["generos"].append(folder_genero)
      #print(f"\nProcesando {folder_genero}")

      #Obtenemos los audios del directorio
      for audio_name in filenames:

        audio_path = os.path.join(dirpath, audio_name)
        signal, sr = librosa.load(audio_path, sr=frecuencia_muestra)


        #Como tenemos pocos datos, vamos a dividir el procesamiento de los
        #audios en diferentes segmentos, entonces para un audio tendriamos
        #varios segmentos del mismo para usar como datos de entrenamiento
        #o testeo.
        for segmento in range(num_segmentos):

          #definimos el inicio y final del actual segmento
          inicio_de_muestra = num_de_muestras_por_segmento * segmento 
          fin_de_muestra = inicio_de_muestra + num_de_muestras_por_segmento

          # Extraemos el mfcc solo del intervalo definido arriba.
          mfcc = librosa.feature.mfcc(signal[inicio_de_muestra: fin_de_muestra],
                                      sr=sr,
                                      n_fft = n_fft,
                                      n_mfcc = n_mfcc,
                                      hop_length = hop_length)

          
          mfcc = mfcc.T

          if len(mfcc) == num_de_vectores_mfcc_por_segmento:
            data["mfcc"].append(mfcc.tolist())
            data["labels"].append(indice - 1)
            #print(f"{audio_path}, segmento:{segmento}")
  
  with open(json_path, "w") as fp:
    json.dump(data, fp, indent=4)



In [16]:
save_mfcc(folder_path, json_path=json_path)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
/content/drive/My Drive/Data/genres_original/blues/blues.00097.wav, segmento:8
/content/drive/My Drive/Data/genres_original/blues/blues.00097.wav, segmento:9

Procesando disco
/content/drive/My Drive/Data/genres_original/disco/disco.00004.wav, segmento:0
/content/drive/My Drive/Data/genres_original/disco/disco.00004.wav, segmento:1
/content/drive/My Drive/Data/genres_original/disco/disco.00004.wav, segmento:2
/content/drive/My Drive/Data/genres_original/disco/disco.00004.wav, segmento:3
/content/drive/My Drive/Data/genres_original/disco/disco.00004.wav, segmento:4
/content/drive/My Drive/Data/genres_original/disco/disco.00004.wav, segmento:5
/content/drive/My Drive/Data/genres_original/disco/disco.00004.wav, segmento:6
/content/drive/My Drive/Data/genres_original/disco/disco.00004.wav, segmento:7
/content/drive/My Drive/Data/genres_original/disco/disco.00004.wav, segmento:8
/content/drive/My Drive/Data/genres_original/d